## Imports

In [127]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from typing import Tuple, Dict
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.18.0


## Hyperparameters - Easily Adjustable

In [128]:
# Model parameters
LOOKBACK_L = 60          # Window size for feature sequences
HORIZON_H = 5           # Forecast horizon (steps ahead)

# Triple barrier parameters
PT_PCT = 0.01           # Profit target: 1.0% (or use dynamic based on volatility)
SL_PCT = 0.010           # Stop loss: 1.0%
USE_DYNAMIC_BARRIERS = True  # Use volatility-based barriers
VOL_MULTIPLIER = 1.5     # Multiplier for volatility-based barriers

# Training parameters
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.001

print(f"Lookback: {LOOKBACK_L}, Horizon: {HORIZON_H}")
print(f"Profit Target: {PT_PCT*100:.1f}%, Stop Loss: {SL_PCT*100:.1f}%")
print(f"Dynamic Barriers: {USE_DYNAMIC_BARRIERS}")

Lookback: 60, Horizon: 5
Profit Target: 1.0%, Stop Loss: 1.0%
Dynamic Barriers: True


## Data Loading & Preprocessing

In [129]:
def load_and_merge_data(eth_path: str, btc_path: str) -> pd.DataFrame:
    """
    Load ETH and BTC data, merge on timestamp with forward fill.
    
    Args:
        eth_path: Path to ETH CSV file
        btc_path: Path to BTC CSV file
    
    Returns:
        Merged DataFrame with ETH and BTC data
    """
    print("Loading data...")
    eth_df = pd.read_csv(eth_path)
    btc_df = pd.read_csv(btc_path)
    
    # Rename 'time' column to 'timestamp' if it exists
    if 'time' in eth_df.columns:
        eth_df = eth_df.rename(columns={'time': 'timestamp'})
    if 'time' in btc_df.columns:
        btc_df = btc_df.rename(columns={'time': 'timestamp'})
    
    # Ensure timestamp columns
    eth_df['timestamp'] = pd.to_datetime(eth_df['timestamp'])
    btc_df['timestamp'] = pd.to_datetime(btc_df['timestamp'])
    
    # Merge on timestamp
    df = pd.merge(eth_df, btc_df, on='timestamp', how='outer', suffixes=('_eth', '_btc'))
    df = df.sort_values('timestamp').reset_index(drop=True)
    
    # Rename close price columns for consistency
    if 'ETH_close' in df.columns:
        df = df.rename(columns={'ETH_close': 'price_eth'})
    if 'BTC_close' in df.columns:
        df = df.rename(columns={'BTC_close': 'price_btc'})
    
    # Rename volume columns if they exist
    if 'ETH_volume' in df.columns:
        df = df.rename(columns={'ETH_volume': 'volume_eth'})
    if 'BTC_volume' in df.columns:
        df = df.rename(columns={'BTC_volume': 'volume_btc'})
    
    # Forward fill missing values
    df = df.fillna(method='ffill').dropna()
    
    print(f"Merged data shape: {df.shape}")
    print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    
    return df

In [130]:
# Load data
ETH_PATH = '../data/specific_asset_data/ETH_data.csv'
BTC_PATH = '../data/specific_asset_data/BTC_data.csv'

df = load_and_merge_data(ETH_PATH, BTC_PATH)
df.head()

Loading data...
Merged data shape: (1046782, 5)
Date range: 2023-11-06 16:31:00+00:00 to 2025-11-10 16:24:00+00:00
Merged data shape: (1046782, 5)
Date range: 2023-11-06 16:31:00+00:00 to 2025-11-10 16:24:00+00:00


,timestamp,price_eth,volume_eth,price_btc,volume_btc
3,2023-11-06 16:31:00+00:00,1898.02,41.677949,35059.11,2.883760
4,2023-11-06 16:32:00+00:00,1897.83,43.216107,35056.92,4.001168
5,2023-11-06 16:33:00+00:00,1898.02,69.148875,35055.26,2.488042
6,2023-11-06 16:34:00+00:00,1897.29,83.485523,35050.93,9.153628
7,2023-11-06 16:35:00+00:00,1899.25,130.998962,35082.45,3.128783


## Feature Engineering

In [131]:
def create_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create robust features for modeling:
    - Log returns for ETH and BTC
    - Rolling volatility (std of returns)
    - Volume change percentage (capped to handle extreme values)
    
    Args:
        df: Input DataFrame with price and volume columns
    
    Returns:
        DataFrame with engineered features
    """
    print("\nCreating features...")
    df = df.copy()
    
    # Log returns
    df['eth_log_return'] = np.log(df['price_eth'] / df['price_eth'].shift(1))
    df['btc_log_return'] = np.log(df['price_btc'] / df['price_btc'].shift(1))
    
    # Rolling volatility (10-period standard deviation of returns)
    df['eth_volatility'] = df['eth_log_return'].rolling(window=10).std()
    df['btc_volatility'] = df['btc_log_return'].rolling(window=10).std()
    
    # Volume change (with safeguards for extreme values)
    # Use robust method: log of ratio instead of pct_change to avoid infinities
    df['eth_volume_change'] = np.log(df['volume_eth'] / df['volume_eth'].shift(1).replace(0, 1e-10))
    df['btc_volume_change'] = np.log(df['volume_btc'] / df['volume_btc'].shift(1).replace(0, 1e-10))
    
    # Cap extreme log volume changes at +/- 2 (roughly 640% up or 86% down)
    df['eth_volume_change'] = df['eth_volume_change'].clip(-2, 2)
    df['btc_volume_change'] = df['btc_volume_change'].clip(-2, 2)
    
    # Drop NaN rows from feature creation
    df = df.dropna().reset_index(drop=True)
    
    print(f"Features created. Shape after cleaning: {df.shape}")
    print(f"Volume change range: ETH [{df['eth_volume_change'].min():.2f}, {df['eth_volume_change'].max():.2f}], "
          f"BTC [{df['btc_volume_change'].min():.2f}, {df['btc_volume_change'].max():.2f}]")
    
    return df

In [132]:
# Create features
df = create_features(df)
df[['timestamp', 'price_eth', 'eth_log_return', 'eth_volatility', 'eth_volume_change']].head()


Creating features...
Features created. Shape after cleaning: (1046772, 11)
Volume change range: ETH [-2.00, 2.00], BTC [-2.00, 2.00]


,timestamp,price_eth,eth_log_return,eth_volatility,eth_volume_change
0,2023-11-06 16:41:00+00:00,1899.10,-0.000216,0.000640,-0.625194
1,2023-11-06 16:42:00+00:00,1899.71,0.000321,0.000643,0.012527
2,2023-11-06 16:43:00+00:00,1899.82,0.000058,0.000643,0.438088
3,2023-11-06 16:44:00+00:00,1899.35,-0.000247,0.000633,-0.698388
4,2023-11-06 16:45:00+00:00,1899.03,-0.000168,0.000546,0.447734


## Triple Barrier Labeling

This is the core logic for generating directional labels:

For each timestamp `t`:
1. Set **vertical barrier** at `t + HORIZON_H` (time limit)
2. Set **top barrier** (profit) at `current_price * (1 + PT)`
3. Set **bottom barrier** (stop) at `current_price * (1 - SL)`
4. Check which barrier is hit first:
   - Top hit first → **Label = 1** (Buy signal)
   - Bottom hit first → **Label = -1** (Sell/Avoid signal)
   - Vertical hit first → **Label = 0** (Neutral)

In [133]:
def get_triple_barrier_labels(
    df: pd.DataFrame,
    price_col: str = 'price_eth',
    horizon: int = HORIZON_H,
    pt_pct: float = PT_PCT,
    sl_pct: float = SL_PCT,
    use_dynamic: bool = USE_DYNAMIC_BARRIERS,
    vol_col: str = 'eth_volatility',
    vol_mult: float = VOL_MULTIPLIER
) -> np.ndarray:
    """
    Implement Triple Barrier Labeling for directional prediction.
    
    For each timestamp t:
    1. Set vertical barrier at t + horizon
    2. Set top barrier (profit) at price * (1 + PT)
    3. Set bottom barrier (stop) at price * (1 - SL)
    4. Check which barrier is hit first:
       - Top hit first → Label = 1 (Buy signal)
       - Bottom hit first → Label = -1 (Sell/Avoid signal)
       - Vertical hit first (time expires) → Label = 0 (Neutral)
    
    Args:
        df: DataFrame with price data
        price_col: Column name for price
        horizon: Forecast horizon (vertical barrier)
        pt_pct: Profit target percentage (fixed)
        sl_pct: Stop loss percentage (fixed)
        use_dynamic: Use volatility-based dynamic barriers
        vol_col: Volatility column for dynamic barriers
        vol_mult: Multiplier for volatility
    
    Returns:
        Array of labels (-1, 0, 1)
    """
    print("\nGenerating Triple Barrier Labels...")
    print(f"  Horizon: {horizon} steps")
    print(f"  Profit Target: {pt_pct*100:.2f}%")
    print(f"  Stop Loss: {sl_pct*100:.2f}%")
    print(f"  Dynamic Barriers: {use_dynamic}")
    
    prices = df[price_col].values
    n = len(prices)
    labels = np.zeros(n, dtype=int)
    
    # Get volatility if using dynamic barriers
    if use_dynamic and vol_col in df.columns:
        volatility = df[vol_col].values
    else:
        volatility = None
    
    for i in range(n - horizon):
        current_price = prices[i]
        
        # Determine barrier widths
        if use_dynamic and volatility is not None:
            vol = volatility[i] if not np.isnan(volatility[i]) else pt_pct
            pt = vol * vol_mult
            sl = vol * vol_mult * 0.7  # Asymmetric: tighter stop
        else:
            pt = pt_pct
            sl = sl_pct
        
        # Set barriers
        top_barrier = current_price * (1 + pt)
        bottom_barrier = current_price * (1 - sl)
        
        # Look ahead over the horizon
        future_prices = prices[i+1:i+1+horizon]
        
        # Find first barrier touch
        hit_top = False
        hit_bottom = False
        
        for future_price in future_prices:
            if future_price >= top_barrier:
                hit_top = True
                break
            elif future_price <= bottom_barrier:
                hit_bottom = True
                break
        
        # Assign label
        if hit_top:
            labels[i] = 1   # Buy signal (profit target hit)
        elif hit_bottom:
            labels[i] = -1  # Sell/Avoid signal (stop loss hit)
        else:
            labels[i] = 0   # Neutral (time barrier hit)
    
    # Last `horizon` samples cannot be labeled → assign neutral
    labels[-horizon:] = 0
    
    # Print label distribution
    unique, counts = np.unique(labels, return_counts=True)
    print("\nLabel Distribution:")
    for label, count in zip(unique, counts):
        label_name = {-1: "Sell/Avoid", 0: "Neutral", 1: "Buy"}[label]
        print(f"  {label_name:12} ({label:2d}): {count:6d} ({count/len(labels)*100:5.2f}%)")
    
    return labels

In [134]:
# Generate labels
labels = get_triple_barrier_labels(
    df,
    price_col='price_eth',
    horizon=HORIZON_H,
    pt_pct=PT_PCT,
    sl_pct=SL_PCT,
    use_dynamic=USE_DYNAMIC_BARRIERS,
    vol_col='eth_volatility',
    vol_mult=VOL_MULTIPLIER
)


Generating Triple Barrier Labels...
  Horizon: 5 steps
  Profit Target: 1.00%
  Stop Loss: 1.00%
  Dynamic Barriers: True

Label Distribution:
  Sell/Avoid   (-1): 464915 (44.41%)
  Neutral      ( 0): 236785 (22.62%)
  Buy          ( 1): 345072 (32.97%)

Label Distribution:
  Sell/Avoid   (-1): 464915 (44.41%)
  Neutral      ( 0): 236785 (22.62%)
  Buy          ( 1): 345072 (32.97%)


## Create Sequences for Time-Series Modeling

In [135]:
def create_sequences(
    df: pd.DataFrame,
    labels: np.ndarray,
    feature_cols: list,
    lookback: int = LOOKBACK_L
) -> Tuple[np.ndarray, np.ndarray, StandardScaler]:
    """
    Create sequences for time-series modeling.
    Filters out neutral labels (0) and keeps only directional signals.
    
    Args:
        df: DataFrame with features
        labels: Array of labels (-1, 0, 1)
        feature_cols: List of feature column names
        lookback: Sequence length
    
    Returns:
        X: Feature sequences (samples, lookback, features)
        y: Binary labels (samples,) - 0 for down, 1 for up
        scaler: Fitted StandardScaler
    """
    print(f"\nCreating sequences with lookback={lookback}...")
    
    # Normalize features
    scaler = StandardScaler()
    features = df[feature_cols].values
    features_scaled = scaler.fit_transform(features)
    
    X, y = [], []
    
    for i in range(lookback, len(features)):
        # Skip neutral labels (0)
        if labels[i] != 0:
            X.append(features_scaled[i-lookback:i])
            y.append(labels[i])
    
    X = np.array(X)
    y = np.array(y)
    
    # Convert labels to binary: -1→0 (down), 1→1 (up)
    y = (y + 1) // 2
    
    print(f"Before balancing: Down (0): {(y==0).sum()}, Up (1): {(y==1).sum()}")
    
    # Balance classes by downsampling the majority class
    down_indices = np.where(y == 0)[0]
    up_indices = np.where(y == 1)[0]
    
    # Downsample to match minority class
    min_samples = min(len(down_indices), len(up_indices))
    
    if len(down_indices) > min_samples:
        down_indices = np.random.choice(down_indices, min_samples, replace=False)
    if len(up_indices) > min_samples:
        up_indices = np.random.choice(up_indices, min_samples, replace=False)
    
    # Combine and shuffle
    balanced_indices = np.concatenate([down_indices, up_indices])
    np.random.shuffle(balanced_indices)
    
    X = X[balanced_indices]
    y = y[balanced_indices]

    print(f"Sequences created: X shape = {X.shape}, y shape = {y.shape}")
    print(f"After balancing: Down (0): {(y==0).sum()}, Up (1): {(y==1).sum()}")
    
    return X, y, scaler    

In [136]:
# Define feature columns
feature_cols = [
    'eth_log_return', 'btc_log_return',
    'eth_volatility', 'btc_volatility',
    'eth_volume_change', 'btc_volume_change'
]

# Create sequences
X, y, scaler = create_sequences(df, labels, feature_cols, lookback=LOOKBACK_L)

print(f"\nFeature sequence shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Number of features: {len(feature_cols)}")


Creating sequences with lookback=60...
Before balancing: Down (0): 464886, Up (1): 345058
Before balancing: Down (0): 464886, Up (1): 345058
Sequences created: X shape = (690116, 60, 6), y shape = (690116,)
After balancing: Down (0): 345058, Up (1): 345058

Feature sequence shape: (690116, 60, 6)
Labels shape: (690116,)
Number of features: 6
Sequences created: X shape = (690116, 60, 6), y shape = (690116,)
After balancing: Down (0): 345058, Up (1): 345058

Feature sequence shape: (690116, 60, 6)
Labels shape: (690116,)
Number of features: 6


## Train/Validation/Test Split (Chronological)

In [137]:
def split_data(
    X: np.ndarray,
    y: np.ndarray,
    train_size: float = 0.7,
    val_size: float = 0.15
) -> Dict[str, np.ndarray]:
    """
    Split data chronologically (no shuffling for time-series).
    
    Args:
        X: Feature sequences
        y: Labels
        train_size: Proportion for training
        val_size: Proportion for validation
    
    Returns:
        Dictionary with train/val/test splits
    """
    n = len(X)
    train_end = int(n * train_size)
    val_end = int(n * (train_size + val_size))
    
    splits = {
        'X_train': X[:train_end],
        'y_train': y[:train_end],
        'X_val': X[train_end:val_end],
        'y_val': y[train_end:val_end],
        'X_test': X[val_end:],
        'y_test': y[val_end:]
    }
    
    print("\nData split (chronological):")
    print(f"  Train: {len(splits['X_train']):6d} samples ({train_size*100:.0f}%)")
    print(f"  Val:   {len(splits['X_val']):6d} samples ({val_size*100:.0f}%)")
    print(f"  Test:  {len(splits['X_test']):6d} samples ({(1-train_size-val_size)*100:.0f}%)")
    
    return splits

In [138]:
# Split data
splits = split_data(X, y, train_size=0.7, val_size=0.15)


Data split (chronological):
  Train: 483081 samples (70%)
  Val:   103517 samples (15%)
  Test:  103518 samples (15%)


## Build CNN-LSTM Model

Architecture:
- **Conv1D layers**: Extract local patterns from sequences
- **MaxPooling1D**: Downsample features
- **LSTM layers**: Capture long-term temporal dependencies
- **Dense layers**: Classification head with softmax (3 classes)

In [139]:
def build_cnn_lstm_model(
    input_shape: Tuple[int, int],
    num_classes: int = 2
) -> keras.Model:
    """
    Build CNN-LSTM model for binary directional prediction.
    
    Architecture:
    - Conv1D: Extract local patterns
    - MaxPooling1D: Downsample
    - LSTM: Capture temporal dependencies
    - Dense: Classification head
    
    Args:
        input_shape: (lookback, n_features)
        num_classes: Number of output classes (2: Down/Up)
    
    Returns:
        Compiled Keras model
    """
    print("\nBuilding CNN-LSTM model...")
    
    model = keras.Sequential([
        # Input layer
        layers.Input(shape=input_shape),
        
        # CNN layers for local feature extraction
        layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        
        layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        
        # Single LSTM layer for temporal dependencies
        layers.LSTM(units=100),
        layers.Dropout(0.3),
        
        # Classification head
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    # Compile model
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [140]:
# Build model
model = build_cnn_lstm_model(input_shape=(LOOKBACK_L, len(feature_cols)))
model.summary()


Building CNN-LSTM model...


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_10 (Conv1D)              │ (None, 60, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 60, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 30, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 30, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 15, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 100)            │        91,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         6,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,882 (487.82 KB)

 Trainable params: 124,498 (486.32 KB)

 Non-trainable params: 384 (1.50 KB)

## Train Model

In [141]:
# Setup callbacks
early_stop = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6
)

print("\nTraining model...")
print(f"Batch size: {BATCH_SIZE}, Max epochs: {EPOCHS}")


Training model...
Batch size: 64, Max epochs: 50


In [142]:
# Train
history = model.fit(
    splits['X_train'], splits['y_train'],
    validation_data=(splits['X_val'], splits['y_val']),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

Epoch 1/50
7549/7549 ━━━━━━━━━━━━━━━━━━━━ 145s 19ms/step - accuracy: 0.5054 - loss: 0.6951 - val_accuracy: 0.5034 - val_loss: 0.6931 - learning_rate: 0.0010
Epoch 2/50
7549/7549 ━━━━━━━━━━━━━━━━━━━━ 145s 19ms/step - accuracy: 0.5054 - loss: 0.6951 - val_accuracy: 0.5034 - val_loss: 0.6931 - learning_rate: 0.0010
Epoch 2/50
7549/7549 ━━━━━━━━━━━━━━━━━━━━ 135s 18ms/step - accuracy: 0.5127 - loss: 0.6927 - val_accuracy: 0.5167 - val_loss: 0.6923 - learning_rate: 0.0010
Epoch 3/50
7549/7549 ━━━━━━━━━━━━━━━━━━━━ 135s 18ms/step - accuracy: 0.5127 - loss: 0.6927 - val_accuracy: 0.5167 - val_loss: 0.6923 - learning_rate: 0.0010
Epoch 3/50
7549/7549 ━━━━━━━━━━━━━━━━━━━━ 133s 18ms/step - accuracy: 0.5133 - loss: 0.6926 - val_accuracy: 0.5186 - val_loss: 0.6920 - learning_rate: 0.0010
Epoch 4/50
7549/7549 ━━━━━━━━━━━━━━━━━━━━ 133s 18ms/step - accuracy: 0.5133 - loss: 0.6926 - val_accuracy: 0.5186 - val_loss: 0.6920 - learning_rate: 0.0010
Epoch 4/50
7549/7549 ━━━━━━━━━━━━━━━━━━━━ 136s 18ms/step -

KeyboardInterrupt: 

## Plot Training History

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Loss
ax1.plot(history.history['loss'], label='Train Loss')
ax1.plot(history.history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training & Validation Loss')
ax1.legend()
ax1.grid(True)

# Accuracy
ax2.plot(history.history['accuracy'], label='Train Accuracy')
ax2.plot(history.history['val_accuracy'], label='Val Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Training & Validation Accuracy')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

## Evaluation on Test Set

In [ ]:
print("="*60)
print("EVALUATION ON TEST SET")
print("="*60)

# Predictions
y_pred_probs = model.predict(splits['X_test'])
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = splits['y_test']

# Accuracy
accuracy = np.mean(y_pred == y_true)
print(f"\nTest Accuracy: {accuracy*100:.2f}%")

In [ ]:
# Per-class metrics
print("\nPer-Class Performance:")
class_names = ['Down/Sell', 'Up/Buy']

for i, class_name in enumerate(class_names):
    mask_true = (y_true == i)
    mask_pred = (y_pred == i)
    
    true_positives = np.sum((y_true == i) & (y_pred == i))
    precision = true_positives / np.sum(mask_pred) if np.sum(mask_pred) > 0 else 0
    recall = true_positives / np.sum(mask_true) if np.sum(mask_true) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    print(f"  {class_name:12}: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}")

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print("              Predicted")
print("              Sell  Neut  Buy")
for i, row in enumerate(cm):
    print(f"  Actual {class_names[i]:7}: {row}")

# Visualize confusion matrix
import seaborn as sns
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Save Model

In [ ]:
# Save model
model.save('eth_direction_model.h5')
print("\n✓ Model saved as 'eth_direction_model.h5'")

# Save scaler
import joblib
joblib.dump(scaler, 'feature_scaler.pkl')
print("✓ Scaler saved as 'feature_scaler.pkl'")

## Summary

This notebook implements a complete directional prediction pipeline:

1. **Triple Barrier Labeling**: Novel labeling method using profit target, stop loss, and time barriers
2. **Feature Engineering**: Log returns, volatility, and volume changes for ETH and BTC
3. **CNN-LSTM Architecture**: Combines local pattern extraction with temporal modeling
4. **Proper Time-Series Handling**: Chronological splits, no data leakage

### Key Hyperparameters:
- `LOOKBACK_L`: Sequence length for features
- `HORIZON_H`: Forecast horizon for labels
- `PT_PCT`, `SL_PCT`: Profit/Stop thresholds
- `USE_DYNAMIC_BARRIERS`: Volatility-based adaptive barriers

### Next Steps:
- Tune hyperparameters using grid search
- Add more features (technical indicators, sentiment, etc.)
- Implement backtesting with the predictions
- Deploy for live trading